In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [2]:
pff = pd.read_csv('pff_pi.csv', index_col=0)

In [3]:
pff.head()

,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PF,PA,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
0,0,Miami Dolphins,Pittsburgh Steelers,2006,1,2006-09-07,8:30pm,A,L,17,28,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-
0,0,Pittsburgh Steelers,Miami Dolphins,2006,1,2006-09-07,8:30pm,H,W,28,17,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-
1,1,Atlanta Falcons,Carolina Panthers,2006,1,2006-09-10,1:00pm,A,W,20,6,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-
1,1,Carolina Panthers,Atlanta Falcons,2006,1,2006-09-10,1:00pm,H,L,6,20,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-
2,2,New Orleans Saints,Cleveland Browns,2006,1,2006-09-10,1:00pm,A,W,19,14,68.4,66.9,58.6,86.2,72.0,50.4,66.0,67.4,81.6,61.8,59.4,57.8,-,66.3,60.9,57.2,68.7,60.7,58.8,57.0,70.5,71.8,71.9,53.2,68.0,-


In [97]:
pff = pd.read_csv('pff_pi.csv', index_col=0)
pff = pff.reset_index().drop(columns='index').reset_index().rename(columns={'index':'game_id'}).set_index('game_id')
pff = pff[['gamenum','Team', 'Opposing Team', 'Season', 'Week', 'Date', 'Time', 'H/A', 'W/L',
       'PA', 'PF', 'OVR', 'OFF', 'PASS', 'PBLK', 'RECV', 'RUN', 'RBLK', 'DEF',
       'RDEF', 'TACK', 'PRSH', 'COV', 'SPEC', 'OVR_opp', 'OFF_opp', 'PASS_opp',
       'PBLK_opp', 'RECV_opp', 'RUN_opp', 'RBLK_opp', 'DEF_opp', 'RDEF_opp',
       'TACK_opp', 'PRSH_opp', 'COV_opp', 'SPEC_opp']]
# pff.iloc[:,9:].drop(columns=['SPEC','SPEC_opp'])
pff.tail(2)

,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PA,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7166,3583,San Francisco 49Ers,Seattle Seahawks,2019,17,12/29/2019,8:20pm,A,W,21,26,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9
7167,3583,Seattle Seahawks,San Francisco 49Ers,2019,17,12/29/2019,8:20pm,H,L,26,21,60.3,70.3,67.1,76.2,67.5,58.9,68.1,39.7,37.0,26.6,61.1,42.6,66.9,79.9,88.9,88.3,81.6,84.6,72.2,69.2,64.7,65.6,66.5,58.7,65.5,56.3


In [98]:
df = pff.copy()

In [99]:
df.iloc[:,10:]

,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,17,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-
1,28,74.2,68.3,65.5,45.2,73.5,60.6,64.1,77.0,68.6,59.9,73.2,72.0,-,65.5,64.7,50.3,65.6,69.1,57.0,65.8,64.4,69.9,67.8,80.2,44.8,-
2,20,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-
3,6,60.7,60.5,40.3,74.0,64.3,62.1,72.0,60.7,68.8,68.6,66.0,51.6,-,78.9,65.6,47.5,63.1,63.9,61.1,73.0,89.8,67.4,78.9,76.3,89.9,-
4,19,68.4,66.9,58.6,86.2,72.0,50.4,66.0,67.4,81.6,61.8,59.4,57.8,-,66.3,60.9,57.2,68.7,60.7,58.8,57.0,70.5,71.8,71.9,53.2,68.0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7163,31,70.8,72.9,67.5,75.5,76.9,63.9,54.0,61.5,86.1,71.9,60.6,53.3,64.8,66.0,67.9,69.0,76.2,67.8,60.9,48.2,59.6,69.8,61.6,59.6,51.6,62.8
7164,34,80.5,71.5,68.3,73.4,69.5,61.5,65.2,78.8,75.8,82.1,69.4,80.4,69.6,60.8,60.6,58.3,64.4,61.8,62.3,48.6,61.4,64.1,55.9,57.7,59.5,52.8
7165,17,60.8,60.6,58.3,64.4,61.8,62.3,48.6,61.4,64.1,55.9,57.7,59.5,52.8,80.5,71.5,68.3,73.4,69.5,61.5,65.2,78.8,75.8,82.1,69.4,80.4,69.6


# Basic regression by single season holdout

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [101]:
# initialize lists to store predictions
test_score_predictions = []
holdout_score_predictions = []
test_game_ids = []
holdout_game_ids = []

# start at 2012 season
current_season = 2012

# loop through seasons
while current_season < 2020:
    print(current_season)

    # create train/test and holdout dataframes dataframes
    prediction_df = df[df['Season']<current_season].iloc[:,10:].drop(columns=['SPEC','SPEC_opp']).tail(1280)
    holdout_df = df[df['Season']==current_season].iloc[:,10:].drop(columns=['SPEC','SPEC_opp'])

    # split y, X arrays
    y = prediction_df.iloc[:,0]
    X = prediction_df.iloc[:,1:]

    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=42)

    # append game_ids from y_test
    test_game_ids.append(y_test.index.values)

    # train model - add cross validate later
    reg = LinearRegression().fit(X_train, y_train)

    # predict on test set
    test_preds = reg.predict(X_test)
    # append test_preds
    test_score_predictions.append(test_preds)

    # grab actual scores from the holdout set
    actual_scores = holdout_df.iloc[:,0]
    # get holdout X dataframe
    X_holdout = holdout_df.iloc[:,1:]

    # predict on holdout set
    holdout_preds = reg.predict(X_holdout)
    # append holdout predictions
    holdout_score_predictions.append(holdout_preds)
    
    # append game_ids from holdout set
    holdout_game_ids.append(holdout_df.index.values)

    current_season+=1

2012
2013
2014
2015
2016
2017
2018
2019


In [102]:
# flatten list of lists
flat_holdout_score_predictions = [item for sublist in holdout_score_predictions for item in sublist]
flat_holdout_game_ids = [item for sublist in holdout_game_ids for item in sublist]

results = pd.DataFrame(data=flat_holdout_score_predictions, index=flat_holdout_game_ids, columns=['predicted_score'])
results_merge = results.merge(df, left_index=True, right_index=True)
results_merge.head(2)

,predicted_score,gamenum,Team,Opposing Team,Season,Week,Date,Time,H/A,W/L,PA,PF,OVR,OFF,PASS,PBLK,RECV,RUN,RBLK,DEF,RDEF,TACK,PRSH,COV,SPEC,OVR_opp,OFF_opp,PASS_opp,PBLK_opp,RECV_opp,RUN_opp,RBLK_opp,DEF_opp,RDEF_opp,TACK_opp,PRSH_opp,COV_opp,SPEC_opp
3072,29.265023,1536,Dallas Cowboys,New York Giants,2012,1,09/05/2012,8:30pm,A,W,17,24,72.1,75.2,84.9,64.1,77.9,68.0,61.5,62.9,74.4,57.9,72.0,50.9,-,64.8,68.9,83.0,49.4,68.1,50.9,61.1,57.9,70.1,89.0,64.5,44.6,-
3073,23.997504,1536,New York Giants,Dallas Cowboys,2012,1,09/05/2012,8:30pm,H,L,24,17,64.8,68.9,83.0,49.4,68.1,50.9,61.1,57.9,70.1,89.0,64.5,44.6,-,72.1,75.2,84.9,64.1,77.9,68.0,61.5,62.9,74.4,57.9,72.0,50.9,-


In [104]:
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(results_merge['predicted_score'].values, results_merge['PF'].values))

6.975992908868027

In [128]:
def predicted_results(s):
    if s < 0:
        x = 'L'
    elif s > 0:
        x = 'W'
    return x

def predicted_win_loss(s1, s2):
    if s1 == s2:
        x = 1
    elif s1 != s2:
        x = 0
    return x

# merge results and keep home team rows
analysis = results_merge.iloc[:,0:12].merge(results_merge[['gamenum','Opposing Team','predicted_score']], 
                                            left_on=['gamenum','Team'], 
                                            right_on=['gamenum','Opposing Team'], 
                                            suffixes=['_home','_away']).drop_duplicates(subset='gamenum', keep="last")

analysis['point_diff'] = analysis['predicted_score_home'] - analysis['predicted_score_away']
analysis['predicted_results'] = analysis['point_diff'].apply(predicted_results)

print(analysis.apply(lambda x: predicted_win_loss(x['W/L'], x['predicted_results']), axis=1).sum() / len(analysis))
analysis.tail(2)

0.85009765625


,predicted_score_home,gamenum,Team,Opposing Team_home,Season,Week,Date,Time,H/A,W/L,PA,PF,Opposing Team_away,predicted_score_away,point_diff,predicted_results
4093,15.430605,3582,New York Giants,Philadelphia Eagles,2019,17,12/29/2019,4:25pm,H,L,34,17,New York Giants,28.965092,-13.534487,L
4095,25.901965,3583,Seattle Seahawks,San Francisco 49Ers,2019,17,12/29/2019,8:20pm,H,L,26,21,Seattle Seahawks,37.855911,-11.953946,L
